In [73]:
!pip install xgboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")


In [61]:
# Load data
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [62]:
# Clean TotalCharges
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.dropna(inplace=True)

In [63]:
# Drop customerID
df.drop('customerID', axis=1, inplace=True)

In [64]:
# Encode target
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

In [65]:
# Identify categorical columns *before* encoding
cat_cols_initial = df.select_dtypes(include=['object']).columns.tolist()

In [66]:
# Encode categorical features and store encoders
le_dict = {}
for col in cat_cols_initial:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le

In [67]:
# Features and target
X = df.drop('Churn', axis=1)
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [68]:
# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [76]:
# Models
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(probability=True),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Train and evaluate all models
results = {}
best_model_name = None # To store the name of the best model
max_accuracy = -1 # To store the maximum accuracy found

print("--- Model Training and Evaluation ---")
for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))
    results[name] = acc
    # Update best model if current accuracy is higher
    if acc > max_accuracy:
        max_accuracy = acc
        best_model_name = name

# Print the best model
print(f"\n✅ Best Model: {best_model_name} with accuracy = {results[best_model_name]:.4f}")

# Assign the final_model based on the best_model_name
final_model = models[best_model_name]

--- Model Training and Evaluation ---

Training Logistic Regression...
Logistic Regression Accuracy: 0.7854
              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1033
           1       0.62      0.49      0.55       374

    accuracy                           0.79      1407
   macro avg       0.73      0.69      0.70      1407
weighted avg       0.77      0.79      0.78      1407


Training Random Forest...
Random Forest Accuracy: 0.7903
              precision    recall  f1-score   support

           0       0.83      0.90      0.86      1033
           1       0.64      0.48      0.55       374

    accuracy                           0.79      1407
   macro avg       0.73      0.69      0.71      1407
weighted avg       0.78      0.79      0.78      1407


Training SVM...
SVM Accuracy: 0.7910
              precision    recall  f1-score   support

           0       0.82      0.91      0.87      1033
           1       0.65      0.46

In [77]:
# Train SVM only (since it was the best model)
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)

SVC(probability=True)

In [78]:

# Predict from custom input
def predict_custom_input(input_dict):
    input_df = pd.DataFrame([input_dict])

    # Ensure all expected columns exist and handle missing ones
    # Use X.columns from the training data for reference
    for col in X.columns:
        if col not in input_df.columns:
            if col in cat_cols_initial: # Use the list of initial categorical columns
                # For categorical columns not in input, use mode from the original, unencoded data
                input_df[col] = df[col].mode()[0]
            else:
                # For numeric columns not in input, use mean or 0 from training data
                input_df[col] = 0 # Or X[col].mean() if appropriate

    # Reorder columns to match the training data
    input_df = input_df[X.columns]

    # Encode categorical features in the input_df
    for col in cat_cols_initial: # Iterate through the initial categorical columns
        if col in input_df.columns and col in le_dict:
            try:
                # Check if the label is known by the encoder
                if input_df[col].iloc[0] not in le_dict[col].classes_:
                    print(f"⚠️ Unknown label '{input_df[col].iloc[0]}' in column '{col}'. Attempting to map to a known class.")
                    # A robust way: map to the most frequent class during training, or a designated "unknown" category if handled by the model
                    # For simplicity, let's use the mode of the original *encoded* training column, or the first class
                    # It's crucial that the value assigned is something the *encoder* can handle.
                    # This implies it should be one of the original classes that the encoder learned from.
                    # Find the index of the most frequent class from the original data that the encoder knows
                    most_freq_class_original = df[col].mode()[0]
                    encoded_most_freq = le_dict[col].transform([most_freq_class_original])[0]
                    input_df[col] = encoded_most_freq
                else:
                    input_df[col] = le_dict[col].transform(input_df[col])
            except Exception as e:
                print(f"⚠️ Encoding error in column '{col}': {e}")
                return "Invalid input", 0.0

    # Ensure all columns are numeric after encoding.
    # This step is critical to catch any remaining non-numeric types before scaling.
    for col in input_df.columns:
        if input_df[col].dtype == 'object':
            # This should ideally not happen if encoding is correct, but as a safeguard:
            print(f"Warning: Column '{col}' is still of object type before scaling. Attempting numeric conversion.")
            input_df[col] = pd.to_numeric(input_df[col], errors='coerce')


    # Drop NaNs that might arise from numeric conversion of *any* column
    # This step is still important in case 'coerce' introduces NaNs from unexpected data.
    input_df.dropna(inplace=True)

    if input_df.empty:
        print("⚠️ Input DataFrame is empty after processing due to NaN values in crucial columns. Check input data and encoding.")
        return "Invalid input", 0.0

    # Finally, scale the data
    input_scaled = scaler.transform(input_df)
    pred = svm_model.predict(input_scaled)[0]
    prob = svm_model.predict_proba(input_scaled)[0][1]
    return "Churn" if pred == 1 else "No Churn", round(prob, 4)


In [71]:
# Example input (change these values as needed)
example_input = {
    'gender': 'Female',
    'SeniorCitizen': 0,
    'Partner': 'Yes',
    'Dependents': 'No',
    'tenure': 12,
    'PhoneService': 'Yes',
    'MultipleLines': 'No',
    'InternetService': 'Fiber optic',
    'OnlineSecurity': 'No',
    'OnlineBackup': 'Yes',
    'DeviceProtection': 'Yes',
    'TechSupport': 'No',
    'StreamingTV': 'No',
    'StreamingMovies': 'No',
    'Contract': 'Month-to-month',
    'PaperlessBilling': 'Yes',
    'PaymentMethod': 'Electronic check',
    'MonthlyCharges': 70.35,
    'TotalCharges': 845.5
}

# Make prediction using the predict_custom_input function
prediction, probability = predict_custom_input(example_input)
print(f"\nPrediction for input: {prediction} (Probability: {probability})")


Prediction for input: Churn (Probability: 0.5957)
